Для обчислення **дискретного косинусного перетворення (DCT)** для заданої матриці, спершу слід визначити матрицю вхідних значень, а потім застосувати формулу перетворення DCT. Розрахунок виконується за допомогою такої формули для 2D DCT:

$$
F(u, v) = \alpha(u) \alpha(v) \sum_{x=0}^{M-1} \sum_{y=0}^{N-1} f(x, y) \cos\left[\frac{\pi (2x+1)u}{2M}\right] \cos\left[\frac{\pi (2y+1)v}{2N}\right],
$$

де:
- ( f(x, y) )— елементи вихідної матриці,
- ( F(u, v) )— елементи результуючої матриці після DCT,
-
$$
( \alpha(u), \alpha(v) = \begin{cases} \frac{1}{\sqrt{M}} & u = 0 \text{ або } v = 0 \ \sqrt{\frac{2}{M}} & u, v \neq 0 \end{cases} ) ,
$$
- ( M, N )— розміри матриці.



In [55]:
!pip install numpy scipy
import numpy as np
from scipy.fftpack import idct, dct

np.set_printoptions(suppress=True, precision=3)

1. Побудуємо вихідну матрицю:


In [56]:
# Вихідна матриця:
N = 12
image_matrix = np.array([
    [N, N + 1, N + 2, N - 1],
    [N + 3, N - 1, N - 2, N],
    [N - 3, N + 2, N + 1, N + 2],
    [N + 1, N + 2, N + 3, N + 1]
])
print(image_matrix)

[[12 13 14 11]
 [15 11 10 12]
 [ 9 14 13 14]
 [13 14 15 13]]


2. Виконаємо обчислення дискретного косинусного перетворення.


In [57]:
# Виконуємо 2D DCT (тип-II, ортогональний DCT):
dct_matrix = dct(dct(image_matrix.T, norm='ortho').T, norm='ortho')


Виводимо результат:

In [58]:
# Виведення результату:
print("Матриця після дискретного косинусного перетворення (DCT):")
print(np.round(dct_matrix, 2))


Матриця після дискретного косинусного перетворення (DCT):
[[50.75 -0.33 -1.25 -0.14]
 [-1.9   1.84  1.03  0.76]
 [ 1.75  0.44 -2.25  1.71]
 [-0.02 -3.24 -3.4  -1.34]]


3. Квантування таблиці

Результуюча матриця ( F(u, v) ) ділиться поелементно на матрицю квантування і округлюється до найближчого цілого числа:
$$
F_{quan}(u, v) = \text{round}\left(\frac{F(u, v)}{\text{quant\_matrix}(u, v)}\right)
$$


In [64]:
# Стандартна 4x4 квантуюча матриця :
quant_matrix = np.array([
    [16, 11, 10, 16],
    [12, 12, 14, 19],
    [14, 13, 16, 24],
    [14, 17, 22, 29]
])

# Квантована матриця:
quantized_matrix = np.round(dct_matrix / quant_matrix)


In [65]:
# Виведення результатів:
print("Матриця після DCT:")
print(np.round(dct_matrix, 2))  # Округлення для читабельності
print("\nКвантована матриця:")
print(quantized_matrix)


Матриця після DCT:
[[50.75 -0.33 -1.25 -0.14]
 [-1.9   1.84  1.03  0.76]
 [ 1.75  0.44 -2.25  1.71]
 [-0.02 -3.24 -3.4  -1.34]]

Квантована матриця:
[[ 3. -0. -0. -0.]
 [-0.  0.  0.  0.]
 [ 0.  0. -0.  0.]
 [-0. -0. -0. -0.]]


4. Відновити матрицю зображення за допомогою зворотного ДКП

In [66]:
# Деквантована матриця (повернення значень DCT-матриці):
dequantized_matrix = quantized_matrix * quant_matrix

# Виконуємо зворотне 2D DCT (IDCT):
restored_matrix = idct(idct(dequantized_matrix.T, norm='ortho').T, norm='ortho')

# Виведення відновленої матриці:
print("Деквантована матриця:")
print(dequantized_matrix)

print("\nВідновлена матриця вихідного зображення:")
print(np.round(restored_matrix))


Деквантована матриця:
[[48. -0. -0. -0.]
 [-0.  0.  0.  0.]
 [ 0.  0. -0.  0.]
 [-0. -0. -0. -0.]]

Відновлена матриця вихідного зображення:
[[12. 12. 12. 12.]
 [12. 12. 12. 12.]
 [12. 12. 12. 12.]
 [12. 12. 12. 12.]]


5. Оцінити втрати якості зображення. Для парних варіантів визначити
середньоквадратичне відхилення, для непарних – відношення сигнал/шум.

### Середньоквадратичне відхилення (MSE)
Середньоквадратичне відхилення оцінює, наскільки значення у вихідній матриці зображення (f(x, y)) відрізняються від відновленої матриці (f_{restored}(x, y)):
$$
MSE = \frac{1}{M \times N} \sum_{x=0}^{M-1} \sum_{y=0}^{N-1} \left(f(x, y) - f_{restored}(x, y)\right)^2,
$$
де:
- (M, N) — розміри матриці зображення.
- (f(x, y)) — вихідна (початкова) матриця зображення.
- (f_{restored}(x, y)) — відновлена матриця зображення.


### Відношення сигнал/шум (SNR)
Для непарних варіантів обчислюється SNR. Воно характеризує співвідношення між потужністю сигналу та потужністю шуму:
$$
SNR = 10 \cdot \log_{10} \left(\frac{\sum_{x=0}^{M-1} \sum_{y=0}^{N-1} f(x, y)^2}{\sum_{x=0}^{M-1} \sum_{y=0}^{N-1} \left(f(x, y) - f_{restored}(x, y)\right)^2}\right),
$$
де:
- Чисельник — потужність сигналу.
- Знаменник — потужність шуму (похибка між оригіналом і відновленим зображенням).


In [67]:
# Середньоквадратичне відхилення (MSE):
def calculate_mse(original, restored):
    mse = np.mean((original - restored) ** 2)
    return mse


# 2. Відношення сигнал/шум (SNR):
def calculate_snr(original, restored):
    signal_power = np.sum(original ** 2)
    noise_power = np.sum((original - restored) ** 2)
    snr = 10 * np.log10(signal_power / noise_power)
    return snr


In [69]:
# Вибір методу залежно від варіанту:
variant = 1  # Заміна на свій варіант: парний чи непарний

if variant % 2 == 0:
    # Обчислення MSE
    mse = calculate_mse(image_matrix, restored_matrix)
    print(f"Середньоквадратичне відхилення (MSE): {mse}")
else:
    # Обчислення SNR
    snr = calculate_snr(image_matrix, restored_matrix)
    print(f"Відношення сигнал/шум (SNR): {snr} дБ")


Відношення сигнал/шум (SNR): 16.941911513458116 дБ
